# Pruebas con la librería _anomalib_
- Importamos librerías

In [60]:
from anomalib.data import MVTecAD
from anomalib.deploy import ExportType
from anomalib.engine import Engine
from anomalib.models import Patchcore

- Creamos el _dataset_

In [54]:
from torchvision.transforms import v2

augmentations = v2.Compose([   
    v2.Pad(50, 255, "edge"),
    # v2.CenterCrop(600),
    v2.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    v2.RandomHorizontalFlip(),
    v2.RandomVerticalFlip()
])

In [59]:
# MVTecAD is a popular dataset for anomaly detection
datamodule = MVTecAD(
    root="./datasets/MVTecAD",  # Path to download/store the dataset
    category="bottle",  # MVTec category to use
    train_batch_size=32,  # Number of images per training batch
    eval_batch_size=32,  # Number of images per validation/test batch
    #augmentations = augmentations,
    train_augmentations=augmentations,
    augmentations=None
)

In [56]:
datamodule.setup(stage = "fit")

In [57]:
mi_DataLoader = datamodule.train_dataloader()
batch = next(iter(mi_DataLoader))

print("Formato del batch: ", batch.image.shape)
print("Tamaño del batch: ", batch.batch_size)

print(datamodule.train_data[0].image.shape)

Formato del batch:  torch.Size([32, 3, 900, 900])
Tamaño del batch:  32
torch.Size([3, 900, 900])


- Inicialización del modelo

In [61]:
# Patchcore is a good choice for beginners
model = Patchcore(
    num_neighbors=6,  # Override default model settings
)

In [ ]:
# Vemos que pre_processor tiene el modelo
pre_processor = model.pre_processor
print(pre_processor.transform)

- Creamos el motor de entrenamiento

In [ ]:
engine = Engine(
    max_epochs=1,  # Override default trainer settings
)

- Entrenamos el modelo

In [ ]:
# This produces a lightning model (.ckpt)
engine.fit(datamodule=datamodule, model=model)

- Testeamos el resultado

In [ ]:
test_results = engine.test(datamodule=datamodule, model=model)

In [ ]:
from anomalib.data import PredictDataset
from pathlib import Path

dataset = PredictDataset(
    path=Path("datasets/MVTecAD/bottle/test/broken_large/000.png"),
    image_size=(256, 256),
)